## Intro 
### 1) 포트폴리오 소개
- wconcept 쇼핑몰에서 유저가 구매를 원하는 상품의 정보를 미리 받아 해당 상품을 추천해주는 기능 제안 

### 2) 문제 제기 
- wconcept 쇼핑몰의 화장품군 상품 노출의 비효율성 : 가장 인기가 많은 5개 내외의 상품만 지속적으로 노출됨 = 다양한 상품이 노출되고 있지 않음.
- wconcept은 최근 화장품 분야로 사업을 대폭 확장중인 것으로 파악되는데, 기대 이상의 효과를 가지기 위해서는 더 정교한 상품 노출이 필요할 것으로 사료됨 
- 다른 상품들과 달리 화장품의 경우 유저의 바람대로 상품을 특정하는 것이 중요함. 따라서 미리 유저의 바람을 파악하고 이에 맞추어 상품을 노출시킨다면 유저의 상품 구매 만족도는 올라가고 화장품 매출 또한 증가할 것으로 파악됨 



### 3) 분석 과정 
#### 데이터의 수집 : 웹크롤링
- 크롤링(1) : 각 화장품의 id, 이름, 브랜드명, 가격, 리뷰수 
- 크롤링(2) : 각 화장품의 리뷰 페이지에서 리뷰를 남긴 유저들의 피부타입과 상품의 발림성, 자극도, 향, 끈적임에 대한 평가항목 

#### 데이터의 분석 : 비지도, 지도학습 머신러닝
- 1) 상품 Clustering : 수집한 유저의 피부타입, 화장품의 발림성, 자극도 등을 KMeans를 통해 총 5개의 유형으로 군집화 
- 2) 추천 알고리즘에 적합한 예측모델 선정 : 위에서 얻은 table을 여러 지도학습 머신러닝에 넣어 가장 적합한 예측 모델 선정 
- 3) 임의의 유저 정보(X)를 넣어 모델 최종 확인 : 임의의 유저를 설정하여 피부타입, 유저가 원하는 상품의 발림성, 자극도 등을 넣어 해당되는 군집의 상품을 랜덤으로 추천 

### 4) 분석 결론 
- 화장품의 경우 많은 유저들이 자신의 피부타입에 맞춰서 구매하고 화장품의 질감이나 자극도 등을 꼼꼼하게 따지는 경향이 강함. 따라서 미리 유저가 자신의 정보를 주게끔 유도해서 그에 맞춰 상품군을 추천한다면 기초화장품군의 판매 증가를 기대해볼 수 있음 


In [1]:
import pandas as pd 
import pymysql
import urllib.request as req 
from bs4 import BeautifulSoup as bs
import sqlite3 
import pandas as pd 
import numpy as np 

In [2]:
#sqlites 모듈 사용
conn = sqlite3.connect('./test.db')
cursor = conn.cursor()

In [3]:
# 크롤링한 데이터를 담을 테이블(1) 생성 
q = '''create table if not exists skincare (
    pdt_id char(50),
    product_name char(50),
    brand_name char(50),
    price int,
    review_num int
);
'''

cursor.execute(q)
conn.commit()

In [4]:
#웹크롤링 (1)
#상품 id, 상품명, 브랜드, 가격, 리뷰수 수집 

url='https://www.wconcept.co.kr/Beauty/001001'
params='?page='

limit = 25


for no in range(0, limit+1) :
    pdt_id=[]
    conn_url=f'{url}{params}{no}'
    
    html=req.urlopen(conn_url)
    soup=bs(html, 'html.parser')
    
    
    # 1)상품ID 
    lists = soup.select('div.thumbnail_list >ul >li')
    for l in lists : 
        pdt_id_find = l.a['href']
        pdt_id_int = int(pdt_id_find.split('/')[-1]) 
        pdt_id.append(pdt_id_int)
    
    
    #1)상품명  
    product_name_find = soup.findAll('div', attrs={'class':'ellipsis'})
    product_name_text=[tag.text for tag in product_name_find]

        
    
    # 2)브랜드 
    brand_name_find = soup.findAll('div', attrs={'class':'brand'})
    brand_name_text=[tag.text for tag in brand_name_find]
    

    
    #3)가격 
    price_find = soup.find_all('span', attrs={'class':'discount_price'})
    price_str = [tag.text for tag in price_find]
    
    
    #4)리뷰수 
    review_box=soup.findAll('div', attrs={'class':'review_box'}) 
    
    for i in range(0, len(product_name_text)) : 
        product_name=product_name_text[i]
        brand_name=brand_name_text[i]
        price=int(price_str[i].replace(',',''))
        
        if len(review_box[i])==1 : 
            review_num=0
           
        elif len(review_box[i])!=1:
            review_find=review_box[i].find('div', attrs={'class':'review_count'}).text[:].replace(',','')
            review_num=int(review_find)
        
        
        sql=f"""
            insert into skincare(pdt_id, product_name,brand_name, price, review_num)
            values ({pdt_id[i]}, "{product_name}", "{brand_name}", {price}, {review_num})
            """
            
        cursor.execute(sql)  
  

conn.commit()

In [5]:
# 크롤링한 데이터를 담을 테이블(2) 생성 
q = '''create table if not exists review_detail (
    pdt_id char(50),
    review_detailquestion char(50),
    response char(50),
    response_percent float
);
'''
cursor.execute(q)
conn.commit()

In [6]:
# 웹크롤링(2) 
# 각 상품마다 리뷰를 남긴 유저의 피부타입, 상품에 대한 유저의 평가항목 수집 

pdt_id=[]
#  총 26페이지의 상품 리뷰 크롤링 
for n in range(1,27) : 
    url = 'https://www.wconcept.co.kr/Beauty/001001?page={}'.format(n)
    html=req.urlopen(url)
    soup=bs(html, 'html.parser')
    
    
    # 각 페이지에 나타나는 90개의 상품 pdt_id 리스트로 저장 
    lists = soup.select('div.thumbnail_list >ul >li')
    for l in lists : 
        pdt_id_find = l.a['href']
        pdt_id_int = int(pdt_id_find.split('/')[-1]) 
        pdt_id.append(pdt_id_int)
        
        
    # 개별 상품 리뷰 수집 
   
# driver = webdriver.Chrome('./chromedriver') 
for m in range(2000, len(pdt_id)) :
    
    url = 'https://www.wconcept.co.kr/Product/{}'.format(pdt_id[m])
    html=req.urlopen(url)
    soup=bs(html, 'html.parser')
    try :       
        pdt_detail = []
        categories=soup.select('div.pdt_review--score')
        for category in categories : 
            question = category.h5.get_text()                  # 평가항목(향은 좋은가요? 등) 
            details=category.select('div.pdt_review--per')
            for detail in details :
                re=detail.strong.get_text()                    # 좋음/보통/좋지않음 
                re_percent=int(detail.em.get_text()[:-1])      # % 제외하고 숫자만  
                pdt_detail.append((pdt_id[m], question, re, re_percent))                
        
        sql = """insert into review_detail()
                 values (%s, %s, %s, %s)
              """
              
        cursor.executemany(sql, pdt_detail)
        
    except :
        pass
        

conn.commit()

In [20]:
#크롤링한 두 테이블을 pdt_id를 기준으로 조인 

q= ''' 
   select s.*, r.review_detailquestion, r.response, r.response_percent
   from skincare s JOIN review_detail r 
   on s.pdt_id=r.pdt_id;
   '''

df=pd.read_sql(q, conn)

In [23]:
df

,pdt_id,product_name,brand_name,price,review_num,review_detailquestion,response,response_percent
0,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,보통,0.0
1,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,보통,0.0
2,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,보통,0.0
3,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,보통,0.0
4,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,보통,0.0
...,...,...,...,...,...,...,...,...
167602,300245550,프로텍팅 베이스 선크림,IWLT,39000,11,향은 좋은가요?,좋지않음,0.0
167603,300245550,프로텍팅 베이스 선크림,IWLT,39000,11,향은 좋은가요?,좋지않음,0.0
167604,300245550,프로텍팅 베이스 선크림,IWLT,39000,11,향은 좋은가요?,좋지않음,0.0
167605,300245550,프로텍팅 베이스 선크림,IWLT,39000,11,향은 좋은가요?,좋지않음,0.0


In [24]:
df['review_detailquestion'].unique().tolist()

['발림성은 좋나요?',
 '어떤 피부타입이세요?',
 '자극도는 어떤가요?',
 '제품에 끈적임이 있나요?',
 '향은 좋은가요?',
 '발색력은 좋은가요?',
 '지속력은 좋은가요?',
 '피부톤이 어떻게 되세요?']

In [25]:
data = df.values.tolist()  
data

[['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징 크림 75ml',
  'VAGHEGGI',
  29900,
  4,
  '발림성은 좋나요?',
  '보통',
  0.0],
 ['301128972',
  '[1+1] 에모지오니 플러스 클렌징

In [26]:
df.groupby(['review_detailquestion','response']).agg({'response_percent':np.mean})

response_percent
review_detailquestion response                  
발림성은 좋나요?             보통               20.101971
                      좋음               79.076264
                      좋지않음              0.647815
발색력은 좋은가요?            보통               18.545455
                      좋음               76.909091
                      좋지않음              4.545455
어떤 피부타입이세요?           건성               36.625107
                      민감성               1.551161
                      복합성              52.246776
                      지성                8.534824
자극도는 어떤가요?            순함               75.673010
                      자극적임              0.995675
                      적당함              22.814014
제품에 끈적임이 있나요?         심함                0.713668
                      없음               61.367647
                      적당함              37.739619
지속력은 좋은가요?            보통                0.000000
                      좋음              100.000000
                      좋지않음              0.000000
피부톤이 어떻게 되세요?         13호              14.285714
                      21호               0.000000
                      23호              28.571429
향은 좋은가요?              보통               37.256920
                      좋음               59.814014
                      좋지않음              2.497405

In [27]:
# 응답항목 수치화, 범주화 작업  

rate_dict = {
    '좋음':2,'보통':1,'좋지않음':0,
    '순함':2,'적당함':1,'자극적임':0,
    '심함':0,'없음':2,
     '건성':'a',
     '지성':'b',
     '복합성':'c',
     '민감성':'d'
}
df['response_re'] = df['response'].map(rate_dict)   #map()함수 
df['response_re']

0         1
1         1
2         1
3         1
4         1
         ..
167602    0
167603    0
167604    0
167605    0
167606    0
Name: response_re, Length: 167607, dtype: object

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167607 entries, 0 to 167606
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   pdt_id                 167607 non-null  object 
 1   product_name           167607 non-null  object 
 2   brand_name             167607 non-null  object 
 3   price                  167607 non-null  int64  
 4   review_num             167607 non-null  int64  
 5   review_detailquestion  167607 non-null  object 
 6   response               167607 non-null  object 
 7   response_percent       167607 non-null  float64
 8   response_re            167418 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 11.5+ MB


In [29]:
#response_re= null, response_percent=0 인 행 삭제  
df['response_percent'].replace(0, np.nan, inplace=True)   #0-> null값으로 
df2=df.dropna(subset=['response_percent', 'response_re'], how='any', axis=0)

#발색력, 피부톤 항목 삭제 -> 색조화장품이 기초화장품으로 잘못 분류된 것으로 보임. 
#분석하고자 하는 데이터는 기초화장품군이므로 색조화장품은 불필요. 
df2=df2[~df2.review_detailquestion.str.contains('발색력')]
df2=df2[~df2.review_detailquestion.str.contains('피부톤')]
df2=df2[~df2.review_detailquestion.str.contains('지속력')]

df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82242 entries, 18 to 167588
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pdt_id                 82242 non-null  object 
 1   product_name           82242 non-null  object 
 2   brand_name             82242 non-null  object 
 3   price                  82242 non-null  int64  
 4   review_num             82242 non-null  int64  
 5   review_detailquestion  82242 non-null  object 
 6   response               82242 non-null  object 
 7   response_percent       82242 non-null  float64
 8   response_re            82242 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 6.3+ MB


In [30]:
#기초화장품과 관련된 질문만 남았는지 확인 
df2.review_detailquestion.unique()

array(['발림성은 좋나요?', '어떤 피부타입이세요?', '자극도는 어떤가요?', '제품에 끈적임이 있나요?',
       '향은 좋은가요?'], dtype=object)

In [31]:
df2.head(20)

,pdt_id,product_name,brand_name,price,review_num,review_detailquestion,response,response_percent,response_re
18,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
19,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
20,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
21,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
22,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
23,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
24,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
25,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
26,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2
27,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2


In [35]:
# rate 컬럼 추가 : 수치항목 -> 점수로, 피부타입 항목 -> a, b, c, d 범주변수로 

df2['response_re_num'] = pd.to_numeric(df2.response_re, errors='coerce')
df2['rate']=df2.response_re_num*df2.response_percent
df2['rate'].fillna(df2.response_re, inplace=True)

In [36]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82242 entries, 18 to 167588
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   pdt_id                 82242 non-null  object 
 1   product_name           82242 non-null  object 
 2   brand_name             82242 non-null  object 
 3   price                  82242 non-null  int64  
 4   review_num             82242 non-null  int64  
 5   review_detailquestion  82242 non-null  object 
 6   response               82242 non-null  object 
 7   response_percent       82242 non-null  float64
 8   response_re            82242 non-null  object 
 9   response_re_num        62865 non-null  float64
 10  rate                   82242 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 7.5+ MB


In [37]:
df2.head(20)

,pdt_id,product_name,brand_name,price,review_num,review_detailquestion,response,response_percent,response_re,response_re_num,rate
18,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
19,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
20,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
21,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
22,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
23,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
24,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
25,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
26,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200
27,301128972,[1+1] 에모지오니 플러스 클렌징 크림 75ml,VAGHEGGI,29900,4,발림성은 좋나요?,좋음,100.0,2,2.0,200


In [62]:
# 위의 테이블에서 분석에 필요한 컬럼들만 따로 추출해서 새로 데이터셋 생성
# 각 화장품의 발림성, 자극도, 끈적임, 향, 리뷰를 남긴 유저들의 피부타입, 화장품의 가격 

df3 = df2.pivot_table(index=['pdt_id'],columns=['review_detailquestion'], values=['rate'], aggfunc='first')
category= ['발림성', '피부타입','자극도','끈적임','향']
df3.columns=category
df3

,발림성,피부타입,자극도,끈적임,향
pdt_id,,,,,
300245550,200,a,200,200,100
300246079,200,a,200,100,200
300248984,200,a,100,200,50
300248985,200,a,100,200,200
300251765,33,a,200,200,33
...,...,...,...,...,...
301118066,200,a,200,200,200
301118154,200,a,200,100,200
301118231,200,a,200,200,200


In [60]:
df3_1 = df2.pivot_table(index=['pdt_id'], values=['price'])
df3_1.head()

,price
pdt_id,
300245550,39000
300246079,45000
300248984,22000
300248985,20000
300251765,49000


In [65]:
df3_2=pd.merge(df3,df3_1,left_on='pdt_id', right_on='pdt_id')
df3_2.rename(columns={'price':'가격'}, inplace=True)
df3_2

,발림성,피부타입,자극도,끈적임,향,가격
pdt_id,,,,,,
300245550,200,a,200,200,100,39000
300246079,200,a,200,100,200,45000
300248984,200,a,100,200,50,22000
300248985,200,a,100,200,200,20000
300251765,33,a,200,200,33,49000
...,...,...,...,...,...,...
301118066,200,a,200,200,200,16800
301118154,200,a,200,100,200,23800
301118231,200,a,200,200,200,4400


In [66]:
df3_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 740 entries, 300245550 to 301128972
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   발림성     738 non-null    object
 1   피부타입    730 non-null    object
 2   자극도     729 non-null    object
 3   끈적임     732 non-null    object
 4   향       731 non-null    object
 5   가격      740 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 40.5+ KB


In [67]:
# 어느 하나라도 결측치가 있는 행 drop 
df3_2=df3_2.dropna(subset=['자극도', '피부타입', '끈적임', '향'], how='any', axis=0)
df3_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 726 entries, 300245550 to 301128972
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   발림성     726 non-null    object
 1   피부타입    726 non-null    object
 2   자극도     726 non-null    object
 3   끈적임     726 non-null    object
 4   향       726 non-null    object
 5   가격      726 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 39.7+ KB


In [68]:
#피부타입 -> get dummies이용하여 encoding

onehot_skintype=pd.get_dummies(df3_2['피부타입'], prefix='skintype')
df4=pd.concat([df3_2, onehot_skintype], axis=1)
df4.drop(['피부타입'], axis=1, inplace=True)
df4.head(20)

,발림성,자극도,끈적임,향,가격,skintype_a,skintype_b,skintype_c,skintype_d
pdt_id,,,,,,,,,
300245550,200,200,200,100,39000,1,0,0,0
300246079,200,200,100,200,45000,1,0,0,0
300248984,200,100,200,50,22000,1,0,0,0
300248985,200,100,200,200,20000,1,0,0,0
300251765,33,200,200,33,49000,1,0,0,0
300251772,200,200,100,200,69000,0,0,1,0
300251801,200,200,134,200,49000,1,0,0,0
300251802,200,200,200,200,49000,0,0,1,0
300278258,40,80,40,100,30000,1,0,0,0


In [69]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 726 entries, 300245550 to 301128972
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   발림성         726 non-null    object
 1   자극도         726 non-null    object
 2   끈적임         726 non-null    object
 3   향           726 non-null    object
 4   가격          726 non-null    int64 
 5   skintype_a  726 non-null    uint8 
 6   skintype_b  726 non-null    uint8 
 7   skintype_c  726 non-null    uint8 
 8   skintype_d  726 non-null    uint8 
dtypes: int64(1), object(4), uint8(4)
memory usage: 36.9+ KB


In [70]:
#자료형 -> int로 
df4[['발림성', '자극도', '끈적임', '향']]=df4[['발림성', '자극도', '끈적임', '향']].astype('int')
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 726 entries, 300245550 to 301128972
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   발림성         726 non-null    int64
 1   자극도         726 non-null    int64
 2   끈적임         726 non-null    int64
 3   향           726 non-null    int64
 4   가격          726 non-null    int64
 5   skintype_a  726 non-null    uint8
 6   skintype_b  726 non-null    uint8
 7   skintype_c  726 non-null    uint8
 8   skintype_d  726 non-null    uint8
dtypes: int64(5), uint8(4)
memory usage: 36.9+ KB


In [107]:
df4.describe()

,발림성,자극도,끈적임,향,가격,skintype_a,skintype_b,skintype_c,skintype_d
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,119.735537,159.542700,130.776860,104.466942,30443.477961,0.663912,0.026171,0.296143,0.013774
std,81.581076,45.753163,59.906247,71.478443,25461.945782,0.472695,0.159753,0.456870,0.116632
min,0.000000,0.000000,0.000000,0.000000,990.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,120.000000,100.000000,45.000000,15375.000000,0.000000,0.000000,0.000000,0.000000
50%,100.000000,176.000000,124.000000,86.000000,24000.000000,1.000000,0.000000,0.000000,0.000000
75%,200.000000,200.000000,200.000000,200.000000,38000.000000,1.000000,0.000000,1.000000,0.000000
max,200.000000,200.000000,200.000000,200.000000,218500.000000,1.000000,1.000000,1.000000,1.000000


In [108]:
df4

,발림성,자극도,끈적임,향,가격,skintype_a,skintype_b,skintype_c,skintype_d
pdt_id,,,,,,,,,
300245550,200,200,200,100,39000,1,0,0,0
300246079,200,200,100,200,45000,1,0,0,0
300248984,200,100,200,50,22000,1,0,0,0
300248985,200,100,200,200,20000,1,0,0,0
300251765,33,200,200,33,49000,1,0,0,0
...,...,...,...,...,...,...,...,...,...
301118066,200,200,200,200,16800,1,0,0,0
301118154,200,200,100,200,23800,1,0,0,0
301118231,200,200,200,200,4400,1,0,0,0


In [109]:
q = '''drop table if exists X_KM;
'''
cursor.execute(q)
conn.commit()

df4.to_sql('X_KM', conn)

## KMeans 를 통해 상품을 5개 유형으로 군집화하기

In [110]:
df_km = pd.read_sql('select * from X_KM;',conn, index_col='pdt_id')
df_km

,발림성,자극도,끈적임,향,가격,skintype_a,skintype_b,skintype_c,skintype_d
pdt_id,,,,,,,,,
300245550,200,200,200,100,39000,1,0,0,0
300246079,200,200,100,200,45000,1,0,0,0
300248984,200,100,200,50,22000,1,0,0,0
300248985,200,100,200,200,20000,1,0,0,0
300251765,33,200,200,33,49000,1,0,0,0
...,...,...,...,...,...,...,...,...,...
301118066,200,200,200,200,16800,1,0,0,0
301118154,200,200,100,200,23800,1,0,0,0
301118231,200,200,200,200,4400,1,0,0,0


In [111]:
#분석을 위한 scaling 
from sklearn import preprocessing 
X = preprocessing.MinMaxScaler().fit(df_km).transform(df_km)

In [112]:
#Kmeans 
from sklearn import cluster 
kmeans = cluster.KMeans(init='k-means++', n_clusters=5, n_init=10)
kmeans.fit(df_km)
cluster_label=kmeans.labels_ 
df_km['Cluster']=cluster_label

In [113]:
df_km.head()

,발림성,자극도,끈적임,향,가격,skintype_a,skintype_b,skintype_c,skintype_d,Cluster
pdt_id,,,,,,,,,,
300245550,200,200,200,100,39000,1,0,0,0,3
300246079,200,200,100,200,45000,1,0,0,0,0
300248984,200,100,200,50,22000,1,0,0,0,3
300248985,200,100,200,200,20000,1,0,0,0,1
300251765,33,200,200,33,49000,1,0,0,0,0


In [114]:
q = '''drop table if exists final;
'''
cursor.execute(q)
conn.commit()

df_km.to_sql('final', conn)

## 머신러닝 분석을 위한 데이터셋 세팅

In [115]:
df_final=pd.read_sql('select * from final;', conn, index_col='pdt_id')


X=df_final.iloc[:, :-1]
y=df_final.iloc[:, -1]

In [116]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 726 entries, 300245550 to 301128972
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   발림성         726 non-null    int64
 1   자극도         726 non-null    int64
 2   끈적임         726 non-null    int64
 3   향           726 non-null    int64
 4   가격          726 non-null    int64
 5   skintype_a  726 non-null    int64
 6   skintype_b  726 non-null    int64
 7   skintype_c  726 non-null    int64
 8   skintype_d  726 non-null    int64
dtypes: int64(9)
memory usage: 56.7+ KB


In [117]:
X.describe()

,발림성,자극도,끈적임,향,가격,skintype_a,skintype_b,skintype_c,skintype_d
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,119.735537,159.542700,130.776860,104.466942,30443.477961,0.663912,0.026171,0.296143,0.013774
std,81.581076,45.753163,59.906247,71.478443,25461.945782,0.472695,0.159753,0.456870,0.116632
min,0.000000,0.000000,0.000000,0.000000,990.000000,0.000000,0.000000,0.000000,0.000000
25%,33.000000,120.000000,100.000000,45.000000,15375.000000,0.000000,0.000000,0.000000,0.000000
50%,100.000000,176.000000,124.000000,86.000000,24000.000000,1.000000,0.000000,0.000000,0.000000
75%,200.000000,200.000000,200.000000,200.000000,38000.000000,1.000000,0.000000,1.000000,0.000000
max,200.000000,200.000000,200.000000,200.000000,218500.000000,1.000000,1.000000,1.000000,1.000000


In [118]:
y.value_counts()/len(y)

1    0.431129
3    0.366391
0    0.150138
4    0.042700
2    0.009642
Name: Cluster, dtype: float64

In [119]:
from sklearn.preprocessing import MinMaxScaler 
scaler_mm = MinMaxScaler() 
X = scaler_mm.fit_transform(X)

In [120]:
# Data Split 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=1,
                                                    stratify=y)

## 머신러닝 모델별 성능평가 

### [1] Ensemble : Voting

In [121]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier

knn_model = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
lr_model = LogisticRegression(C=1.0, class_weight='balanced', multi_class='ovr', n_jobs=-1)
dt_model = DecisionTreeClassifier(random_state=11, max_depth=3, class_weight='balanced')

In [122]:
from sklearn.ensemble import VotingClassifier 
ensemble_model = VotingClassifier(estimators=[('knn', knn_model),
                                             ('lr', lr_model),
                                             ('dt', dt_model)],
                                 voting='soft',
                                 n_jobs=-1)

In [123]:
# train data Score 
train_scores=[m.fit(X_train, y_train).score(X_train, y_train)
             for m in [knn_model, lr_model, dt_model, ensemble_model]]
print(f'train score : \n {train_scores}')


# test data Score 
test_scores=[m.fit(X_test, y_test).score(X_test, y_test)
             for m in [knn_model, lr_model, dt_model, ensemble_model]]
print(f'test score : \n {test_scores}')

train score : 
 [0.743103448275862, 0.7103448275862069, 1.0, 1.0]
test score : 
 [0.6917808219178082, 0.5821917808219178, 1.0, 1.0]


### [2] Ensemble : Random Forest

In [124]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=5,
    random_state=1,
    n_jobs=-1).fit(X_train, y_train)

print(f'train score : {model.score(X_train, y_train)}')
print(f'test score : {model.score(X_test, y_test)}')

train score : 0.9724137931034482
test score : 0.9657534246575342


### [3] XGBoost

In [125]:
import xgboost as xgb 
xgb_model = xgb.XGBClassifier(
            n_estimators=10000,
            max_depth=5,
            subsample=0.5,
            reg_alpha=10,
            random_state=1).fit(X_train, y_train)


print(f'train score : {xgb_model.score(X_train, y_train)}')
print(f'test score : {xgb_model.score(X_test, y_test)}')

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:17:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
train score : 0.9896551724137931
test score : 0.9931506849315068


## 모델의 선택  : Decision Tree or VotingClassifier 

[1]~[3]의 앙상블 모델 중 Decision tree 나 voting classifier클래스가 가장 성능이 좋은 것으로 판단됨.

## 새로운 임의의 X 데이터 생성 후 모델 시험

In [126]:
def recommend(n) : 
    rec_pdt_id=[]
    for n in pred : 
        rec_pdt_id = y.index[y==n]

    rec_pdt_one=np.random.choice(rec_pdt_id, 1)
    print(f'다음 상품은 어떤가요? : {rec_pdt_one}')

#유저가 각 설문 항목에 대해 자신이 원하는 정보를 입력하면 array배열로 변환하고 해당 군집의 상품을 랜덤으로 하나 추출 
# 발림성, 자극도, 끈적임, 향, 가격, 건성, 지성, 복합성, 민감성 
X_temp1=np.array([200,100,100,100, 30000, 1, 0, 0, 0]).reshape(1,-1)
pred=ensemble_model.predict(X_temp1)

recommend(pred)

다음 상품은 어떤가요? : ['300659318']
